# Unit testing of DAGs

Un aspect foarte important în momentul în care se lucrează la anumite proiecte, reprezintă partea de unit test. Acest lucru ne ajută să ne asigurăm că aplicația/pipeline-ul nu s-a stricat după ce am făcut anumite modificări în cod

Există 5 categorii de teste pe care le putem adăuga atunci când lucrăm cu Airflow

1. DAG Validation Test

- În aceste teste verificăm dacă DAG-ul este valid, dacă nu există bucle, putem verifica argumentele 

2. DAG/Pipeline Definition Tests

- putem verifica numărul total de task-uri, dependințele de task-uri au fost setate corect. Scopul nu este să verifice logica DAG-ului, ci doar să verifice dacă modificările care au fost făcute au fost intenționate.

3. Unit Tests

- Pentru a testa logica din anumite funcții externe sau operatori custom

4. Integration Tests

- Interacțiunea dintre două sau mai multe task-uri este testată, dacă task-uri pot schimba date între ele, dacă conexiunile cu surse externe sunt disponibile

5. End to end pipeline test

- Testează toată logica pipeline-ului, verifică dacă outputul unui DAG este corect, verifică performanța (dacă un DAG nu rulează prea încet)

În continuare o să trecem prin primele 2 categorii de testare ca să ne testăm validarea și definirea DAG-ului pe care l-am creat în secțiunea precedentă. Ca să facem asta, în folderul de mnt o să ne creem un director pe care o să îl denumim test_dags

Primul lucru pe care trebuie să îl facem, este să ne creem un fișier denumit "conftest.py" în care să trecem următorul cod

In [ ]:
import pytest
from airflow.models import DagBag

@pytest.fixture(scope="session")
def dagbag():
    return DagBag()

După ce avem acest fișier putem să trecem la partea în care ne testăm DAG-urile. Pentru testare o să ne folosim de "pytest" și o să creem test cu ajutorul claselor. Primul test pe care o să îl creem este acela prin care se verifică dacă toate DAG-urile au fost importate cu succes de către Airflow.

Pentru asta, în cadrul dicetorului de test_dags o să creem un nou fișier denumit "test_dag_validation.py" în care o să introducem următorul cod.

In [ ]:
import pytest
from airflow.models import DagBag

class TestDagValidation:

    def test_import_dags(self, dagbag):
        """
            Verify that Airflow is able to import all DAGs
            in the repo
            - check for typos
            - check for cycles
        """
        assert len(dagbag.import_errors) == 0, "DAG failures detected! Got: {}".format(
            dagbag.import_errors
        )

Mai sus avem codul prin care putem să testăm dacă DAG-urile au fost importate cu succes de către Airflow. Pentru a rula acest test, trebuie să intrăm in interiorul containerului de airflow, să navigăm în directorul unde există acest fișier (/opt/airflow/test_dags) și să rulăm acest fișier cu comanda următoare:

In [ ]:
pytest test_dag_validation.py -v

Atunci când rulăm comanda de mai sus ar trebui să avem următorul rezultat:

<img src="../../ss/airflow-section-04/section-04-ss-01.png">

După cum se poate observa, testul respectiv a trecut. Asta înseamnă că toate DAG-urile care există au fost importate cu succes. În continuare o să creem un nou test prin care o să verificăm dacă DAG-urile respective au email-ul care trebuie. Ca să verificăm asta, în clasa de TestDagValidation o să ne creem o variabilă de clasă care o să aibă ca și valoare email-ul ce trebuie verificat

In [ ]:
import pytest
from airflow.models import DagBag

class TestDagValidation:

    REQUIRED_EMAIL = "georgiuandrei05@gmail.com"

    def test_import_dags(self, dagbag):
        """
            Verify that Airflow is able to import all DAGs
            in the repo
            - check for typos
            - check for cycles
        """
        assert len(dagbag.import_errors) == 0, "DAG failures detected! Got: {}".format(
            dagbag.import_errors
        )

    def test_default_args_email(self, dagbag):
        """
            Verify that DAGs have the required email
            - Check email
        """
        for dag_id, dag in dagbag.dags.items():
            emails = dag.default_args.get('email', [])
            assert self.REQUIRED_EMAIL in emails, "The mail {0} for sending alerts is missing from the DAG {1}".format(self.REQUIRED_EMAIL, dag_id)
            

Dacă rulăm aceste teste o să avem următoarele rezultate.

<img src="../../ss/airflow-section-04/section-04-ss-02.png">

Acum avem un test care este cu fail. Acest test ne spune faptul că mail-ul respectiv lipsește din DAG-ul "test_forex_data_pipeline". Dacă adăugăm argumentele default (unde trecem mail-ul), acest test o să treacă după.

Ce anume mai putem să verificăm este numărul total de task-uri și dacă dependința de task-uri este creată precum trebuie.

Cam cu asta se ocupă parte de testare pentru validare și definiție a task-urilor. Restul de teste o să le parcurgem în cadrul acestui curs.